### Imports

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings

# Suppress specific SHAP TensorFlow warnings
warnings.filterwarnings("ignore", message="Your TensorFlow version is newer than 2.4.0 and so graph support has been removed in eager mode")

# Your code here...

In [3]:
!pip install scikeras

In [4]:
!pip install git+https://github.com/AnotherSamWilson/fastshap.git

  Cloning https://github.com/AnotherSamWilson/fastshap.git to /tmp/pip-req-build-egi51n2o
  Running command git clone --filter=blob:none --quiet https://github.com/AnotherSamWilson/fastshap.git /tmp/pip-req-build-egi51n2o
  Resolved https://github.com/AnotherSamWilson/fastshap.git to commit 2a791a321909143da5606fb3cb092434d413850d
  Preparing metadata (setup.py) ... done


In [5]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam
from scikeras.wrappers import KerasClassifier

In [6]:
!pip install modAL-python

In [7]:
from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [8]:
#!pip install shap

In [9]:
!pip install scikit-optimize

In [10]:
!pip install tensorflow

### Generic Active Learning Model

In [11]:
!pip install evidential-deep-learning

In [12]:
import evidential_deep_learning as edl

def evidential_loss(true, pred):
    alpha = tf.nn.relu(pred) + 1  # Convert evidence to Dirichlet parameters
    return edl.losses.Dirichlet_SOS(true, alpha, t = None)

def create_clf_evidential_model(nodes, n_features, n_classes):
  model = Sequential()
  model.add(Input(shape=(n_features,)))
  model.add(Dense(nodes, activation='relu'))
  model.add(Dense(n_classes, activation='relu'))
  model.compile(optimizer=Adam(), loss=evidential_loss)
  return model

In [84]:
def EvidentialRegressionLoss(true, pred):
        return edl.losses.EvidentialRegression(true, pred, coeff=1e-2)

def create_reg_model(nodes, n_features):
    # Define the model with evidential output for regression
    model = Sequential()
    model.add(Input(shape=(n_features,)))  # Use Input layer for 20 features
    model.add(Dense(nodes, activation='relu', name = 'hidden_layer'))  # 64 units hidden layer
    model.add(edl.layers.DenseNormalGamma(1))  # Output layer for binary classification with sigmoid activation
    model.compile(optimizer=Adam(), loss=EvidentialRegressionLoss, metrics=['accuracy'])

    return model

In [13]:
from tensorflow.keras.layers import Input

def create_clf_model(nodes, n_features):
    model = Sequential()
    model.add(Input(shape=(n_features,)))  # Use Input layer for 20 features
    model.add(Dense(nodes, activation='relu', name = 'hidden_layer'))  # 64 units hidden layer
    model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification with sigmoid activation
    model.compile(optimizer=Adam(), loss="binary_crossentropy", metrics=['accuracy'])
    return model

In [14]:
def create_multi_clf_model(nodes, n_features, n_classes):
    model = Sequential()
    model.add(Input(shape=(n_features,)))  # Use Input layer for 20 features
    model.add(Dense(nodes, activation='relu', name = 'hidden_layer'))  # 64 units hidden layer
    model.add(Dense(n_classes, activation='softmax'))  # Output layer for binary classification with sigmoid activation
    model.compile(optimizer=Adam(), loss="sparse_categorical_crossentropy", metrics=['accuracy'])
    return model

In [16]:
# Step 2: Create a function to initialize ActiveLearner using modAL
def create_active_learner(X_train, y_train, keras_model, query_strategy):

    # Initialize the ActiveLearner
    learner = ActiveLearner(
        estimator=keras_model,
        X_training=X_train,
        y_training=y_train,
        query_strategy=query_strategy  # Use uncertainty sampling
    )

    return learner

In [93]:
import numpy as np
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from tensorflow.keras.losses import sparse_categorical_crossentropy
from sklearn.metrics import mean_squared_error

# Step 3: Active learning loop with custom stopping criteria (Stabilizing Predictions) and n_queries support
def active_learning_loop(learner, X_pool, y_pool, X_stop_set, y_stop, X_sample, y_sample, task = "bin_clf", n_queries=10, stability_threshold=0.01, max_iterations=100, beta = 0.9):
    min_loss = float('inf')  # Initialize to a very large value (since we want to minimize the loss)
    iterations = 0
    streak = 0

    while iterations < max_iterations and len(X_pool) >= n_queries:
        if iterations % 10 == 0:
          print(f"Iteration {iterations + 1}: Current Loss = {min_loss:.4f}, Min Loss = {min_loss:.4f} with pool {len(X_pool)}")


        if learner.query_strategy == conflicting_evidence_strategy_clf:
            query_idx, query_instance = learner.query(X_pool, n_instances=n_queries, task = task)

            # Teach the model using the queried instances
            learner.teach(X_pool[query_idx], y_pool[query_idx])

            # Remove queried instances from the pool
            X_pool = np.delete(X_pool, query_idx, axis=0)
            y_pool = np.delete(y_pool, query_idx, axis=0)

        elif learner.query_strategy == conflicting_evidence_strategy_reg:
            query_idx, query_instance = learner.query(X_pool, n_instances=n_queries, task = task)

            # Teach the model using the queried instances
            learner.teach(X_pool[query_idx], y_pool[query_idx])

            # Remove queried instances from the pool
            X_pool = np.delete(X_pool, query_idx, axis=0)
            y_pool = np.delete(y_pool, query_idx, axis=0)
        else:
            # Need to query with the full dataset which is in sample
            query_idx, query_instance = learner.query(X_sample, n_instances=n_queries, task = task, beta = beta)

            # Manual of the teach function
            X_pool = query_instance
            y_pool = y_sample[query_idx]
            learner.X_training = X_pool
            learner.y_training = y_pool
            learner.estimator.fit(X_pool, y_pool)

        if learner.query_strategy == conflicting_evidence_strategy_clf:

            # Evaluate the learner's predictions (probabilities) on the stop set
            if task == "bin_clf":
              y_pred = learner.predict_proba(X_stop_set)
              current_loss = evidential_loss(y_stop, y_pred)

            elif task == "multi_clf":
              y_pred = learner.predict_proba(X_stop_set)
              current_loss = evidential_loss(y_stop, y_pred)

        elif learner.query_strategy == conflicting_evidence_strategy_reg:
              y_stop_pred = learner.estimator.model.predict(X_stop_set, verbose = 0)
              y_stop_pred[:, 2] += 1
              current_loss = EvidentialRegressionLoss(y_stop, y_stop_pred)

        else:
            if task == "bin_clf":
              y_pred = learner.predict_proba(X_stop_set)
              current_loss = log_loss(y_stop, y_pred)

            elif task == "multi_clf":
              y_pred = learner.predict_proba(X_stop_set)
              current_loss = np.mean(sparse_categorical_crossentropy(y_stop, y_pred))

            elif task == "reg":
              y_pred = learner.predict(X_stop_set)
              current_loss = mean_squared_error(y_stop, y_pred)

            else:
              raise ValueError("Invalid task specified. Must be 'bin_clf', 'multi_clf', or 'reg'.")

        #print(f"Iteration {iterations + 1}: Current Loss = {current_loss:.4f}, Min Loss = {min_loss:.4f} with pool {len(X_pool)}")

        # Check if current loss is better (lower) than the minimum observed so far
        if current_loss < min_loss:
            min_loss = current_loss
            streak = 0  # Reset the streak when the loss improves
        elif streak == 3:  # Stop if the loss hasn't improved for 5 consecutive iterations
            print(f"Iteration {iterations + 1}: Current Loss = {current_loss:.4f}, Min Loss = {min_loss:.4f}")
            break
        else:
            streak += 1  # Increment the streak if there's no improvement

        iterations += 1

    print(f"Active learning stopped after {iterations} iterations with loss {min_loss:.4f}.")

    return min_loss

In [76]:
from sklearn.model_selection import KFold
from sklearn.datasets import make_regression
from scikeras.wrappers import KerasRegressor, KerasClassifier
from tensorflow.keras.callbacks import EarlyStopping

# Step 4: Main function to use a stop set and custom stopping criterion
def main(query_strategy, X = None, y = None, task = "bin_clf", nodes = 64, n_queries = 25, stability_threshold=0.01, n_folds=5, init_size = 0.05, beta = 0.9):
    if query_strategy == sensitivity_strategy:
      print(f"\nNew training with nodes {nodes} and beta {beta}")
    else:
      print(f"\nNew training with nodes {nodes} and n_queries {n_queries}")

    X = np.array(X)
    y = np.array(y)

    # Standardize the dataset
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Perform 5-fold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    # List to store trained models
    models = []
    losses = []

    # Loop through each fold
    for fold_idx, (train_idx, stop_idx) in enumerate(kf.split(X)):
        print(f"Processing Fold {fold_idx + 1}/{n_folds}...")

        # Create training and stop sets for this fold
        X_train, X_stop_set = X[train_idx], X[stop_idx]
        y_train, y_stop = y[train_idx], y[stop_idx]

        sample_idx = np.random.choice(len(X_train),
                                      size=int(init_size * len(X_train)),
                                      replace=False)

        ce_strat = query_strategy == conflicting_evidence_strategy_clf or query_strategy == conflicting_evidence_strategy_reg

        if ce_strat:
            X_sample = X_train[sample_idx]
            y_sample = y_train[sample_idx]
            X_train = np.delete(X_train, sample_idx, axis=0)
            y_train = np.delete(y_train, sample_idx, axis=0)

        elif query_strategy == sensitivity_strategy:
            #Starts with the full set
            X_sample = X_train
            y_sample = y_train

        # Define EarlyStopping callback
        early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)



        if ce_strat :
            if task == "bin_clf":
                #Reshape ys
                n_classes = len(np.unique(y))
                y_sample = tf.keras.utils.to_categorical(y_sample, num_classes=n_classes)
                y_train = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
                y_stop = tf.keras.utils.to_categorical(y_stop, num_classes=n_classes)

                model = create_clf_evidential_model(nodes, n_features = X.shape[1], n_classes = n_classes)
                keras_model = KerasClassifier(model=model, epochs=10, batch_size=32, verbose=0,
                                              validation_split=0.2, callbacks=[early_stopping])

            elif task == "multi_clf":
                #Reshape ys
                n_classes = len(np.unique(y))
                y_sample = tf.keras.utils.to_categorical(y_sample, num_classes=n_classes)
                y_train = tf.keras.utils.to_categorical(y_train, num_classes=n_classes)
                y_stop = tf.keras.utils.to_categorical(y_stop, num_classes=n_classes)


                model = create_clf_evidential_model(nodes, n_features = X.shape[1], n_classes = n_classes)
                keras_model = KerasClassifier(model=model, epochs=10, batch_size=32, verbose=0,
                                              validation_split=0.2, callbacks=[early_stopping])

            elif task == "reg":
                model = create_reg_model(nodes=nodes, n_features=X.shape[1])
                keras_model = KerasRegressor(model=model, epochs=10, batch_size=32, verbose=0,
                                            validation_split=0.2, callbacks=[early_stopping])
            else:
                raise ValueError("Invalid task specified. Must be 'bin_clf', 'multi_clf', or 'reg'.")


        else:
            if task == "bin_clf":
                model = create_clf_model(nodes=nodes, n_features=X.shape[1])
                keras_model = KerasClassifier(model=model, epochs=10, batch_size=32, verbose=0,
                                              validation_split=0.2, callbacks=[early_stopping])

            elif task == "multi_clf":
                model = create_multi_clf_model(nodes=nodes, n_features=X.shape[1], n_classes = len(np.unique(y)))
                keras_model = KerasClassifier(model=model, epochs=10, batch_size=32, verbose=0,
                                              validation_split=0.2, callbacks=[early_stopping])

            elif task == "reg":
                model = create_reg_model(nodes=nodes, n_features=X.shape[1])
                keras_model = KerasRegressor(model=model, epochs=10, batch_size=32, verbose=0,
                                            validation_split=0.2, callbacks=[early_stopping])

            else:
                raise ValueError("Invalid task specified. Must be 'bin_clf', 'multi_clf', or 'reg'.")


        # Create the active learner with the provided query strategy
        learner = create_active_learner(X_sample, y_sample, keras_model, query_strategy)

        # Run the active learning loop with custom stop set criterion
        min_loss = active_learning_loop(learner, X_train, y_train, X_stop_set, y_stop, X_sample, y_sample, task = task, n_queries=n_queries, stability_threshold=stability_threshold, beta = beta)

        # Save the trained learner (model) for this fold
        models.append(learner.estimator)
        losses.append(min_loss)

    avg_loss = np.mean(losses)
    print(f"Average Loss: {avg_loss:.4f}")

    # Return all trained models (one per fold)
    return avg_loss

### Query Strategy

#### ALUS

In [19]:
import numpy as np
import fastshap
import time

def conflicting_evidence_strategy_clf(learner, X_pool, task = "bin_clf", n_instances=10, max = True):
    classifier = learner.estimator.model_
    preds = classifier.predict(X_pool, verbose = 0)
    alpha = np.maximum(preds, 0) + 1  # Convert output to evidence

    evidence_scores = []
    orig_ind = []

    # Get total evidence for when more than one evidence value
    for i in range(len(alpha)):
        sorted_evidence = np.sort(alpha[i])
        evidence_score = sorted_evidence[-1] * sorted_evidence[-2]  # Multiply top two highest evidence values

        if sorted_evidence[-2] > 1.0:
          orig_ind.append(i)
          evidence_scores.append(evidence_score)

    # Sort the evidence scores
    num_scores = len(evidence_scores)
    #print(f"Length of evidence scores is {num_scores}")

    if num_scores == 0:
      evidence_scores = np.max(alpha, axis = 1)
      inds = np.argsort(evidence_scores)[:n_instances]  # Sort in descending order of evidence scores
      return inds, X_pool[inds]

    evidence_scores = np.array(evidence_scores)

    inds = []
    if max == True:
        sorted_indices = np.argsort(evidence_scores)[::-1]  # Sort in descending order of evidence scores
        sorted_orig = np.array(orig_ind)[sorted_indices]
        sorted_evidence_scores = evidence_scores[sorted_indices]

        for i in range(min(n_instances, num_scores)):
          inds.append(sorted_orig[i])
          #print(sorted_evidence_scores[i])
    else:
        sorted_indices = np.argsort(evidence_scores)  # Sort in descending order of evidence scores
        sorted_orig = np.array(orig_ind)[sorted_indices] #Orders references indicies
        sorted_evidence_scores = evidence_scores[sorted_indices] #Gets the score

        for i in range(num_scores - n_instances, num_scores):
          inds.append(sorted_orig[i])
          #print(sorted_evidence_scores[i])

    inds = np.array(inds)
    #print(f"Inds are {inds}")

    # Return the indices and the corresponding instances
    return inds, X_pool[inds]

In [87]:
def conflicting_evidence_strategy_reg(learner, X_pool, task = "reg", n_instances=10, max = True):
    regressor = learner.estimator.model_
    preds = regressor.predict(X_pool, verbose = 0)

    # Just using alpha because measure of uncertainty of variance which is where problems are
    alpha = np.array(preds[:, 2])
    sorted_indices = np.argsort(alpha)[:n_instances]

    # Return the indices and the corresponding instances
    return sorted_indices, X_pool[sorted_indices]

#### Sensitivity

- **Computing Sensitivity Matrix**
$$
S_{oz,ki}^{(p)} = (1 - o_k^{(p)}) o_k^{(p)} \sum_{j=1}^{J} w_{kj} (1 - y_j^{(p)}) y_j^{(p)} v_{ji}
$$

- **Computing Output Sensitivity Vector**
$$
\vec{S}_o^{(p)} = \| S_{oz}^{(p)} \|
$$

- **Computing Informativeness**
$$
\Phi_{\infty}^{(p)} = \| \vec{S}_o^{(p)} \|_{\infty} = \max_k \left\{ |S_{o,k}^{(p)}| \right\}
$$

- **Computing Average Pattern Informativeness**
$$
\overline{\Phi}_\infty = \frac{1}{P_C} \sum_{p=1}^{P_C} \Phi_{\infty}^{(p)}
$$

- **Applying Selection Operator**
$$
A(D_C, F(D_T; W)) = \{ p \in D_C \ | \ \Phi_{\infty}^{(p)} > \psi(\vec{\Phi}_\infty) \}
$$

- **Threshold Function**
$$
\psi(\vec{\Phi}_\infty) = (1 - \beta) \overline{\Phi}_\infty
$$


In [20]:
from tensorflow.keras.models import Model

# Define a function that will return the hidden layer's output
def get_hidden_layer_output(model, X_pool):

    # Extract the hidden layer by name
    input = model.get_layer('hidden_layer').input
    hidden_layer = model.get_layer('hidden_layer').output

    # Create a new model that outputs the hidden layer
    intermediate_layer_model = Model(inputs=input, outputs=hidden_layer)

    # Get the activations for the input X_pool
    hidden_layer_activations = intermediate_layer_model.predict(X_pool, verbose = 0)

    return hidden_layer_activations


In [21]:
# Function to compute sensitivity matrix S(p)_oz
def compute_sensitivity_matrix(X_pool, learner):
    model = learner.estimator.model_

    # Get weights between input-hidden and hidden-output layers
    W_hidden_output = model.layers[-1].get_weights()[0]  # shape (hidden_units, output_units)
    V_input_hidden = model.layers[0].get_weights()[0]    # shape (input_units, hidden_units)

    # Get activations for the hidden layer and output layer
    hidden_layer_activations = get_hidden_layer_output(model, X_pool)
    output_layer_activations = model.predict(X_pool, verbose=0, batch_size=32)  # output layer predictions

    # Getting shapes
    input_shape = V_input_hidden.shape[0]
    hidden_shape = W_hidden_output.shape[0]
    output_shape = W_hidden_output.shape[1]

    # Calculate sensitivity matrix S(p)_oz for each pattern
    sensitivity_matrix = []
    for idx, p in enumerate(X_pool):
        o_p = output_layer_activations[idx]  # output activations for pattern p
        y_p = hidden_layer_activations[idx]  # hidden layer activations for pattern p

        # Equation (4) for sensitivity matrix computation
        S_p_oz = np.zeros((input_shape, output_shape))  # shape (input_units, output_units)
        for k in range(output_shape):  # output units
            for i in range(input_shape):  # input units
                # Summation over hidden units j
                summation = sum(W_hidden_output[j, k] * y_p[j] * (1 - y_p[j]) * V_input_hidden[i, j] for j in range(W_hidden_output.shape[0]))
                S_p_oz[i, k] = o_p[k] * (1 - o_p[k]) * summation
        sensitivity_matrix.append(S_p_oz)

    return np.array(sensitivity_matrix)

In [22]:
# Sensitivity-based query strategy
def sensitivity_strategy(learner, X_pool, task = "bin_clf", beta=0.9, n_instances=None):
    # Step 1: Initialize empty list to store informativeness values
    informativeness_values = []

    # Step 2: For each pattern in the pool (X_pool)
    S_oz_all = compute_sensitivity_matrix(X_pool, learner) #Shape is input by output

    for p in range(X_pool.shape[0]):
        # Step 2.1: Compute output sensitivity vector S(p)_o
        S_p_o = np.linalg.norm(S_oz_all[p], axis=0)

        # Step 2.2: Compute informativeness for pattern p
        Phi_p = np.max(np.abs(S_p_o))

        # Step 2.3: Store informativeness value
        informativeness_values.append(Phi_p)

    # Step 3: Compute the average pattern informativeness over all patterns (Equation 12)
    avg_informativeness = np.mean(informativeness_values)

    # Step 4: Apply the selection threshold using beta (Equation 9)
    threshold = (1 - beta) * avg_informativeness

    # Step 5: Select the most informative patterns (informativeness > threshold)
    selected_indices = [i for i, Phi_p in enumerate(informativeness_values) if Phi_p > threshold]

    return selected_indices, X_pool[selected_indices]


### Testing

#### Normal NN

In [35]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.datasets import make_classification
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical

# Function to create a simple classification model with a single hidden layer
def create_passive_clf_model(nodes, n_features, n_classes=2):
    model = tf.keras.Sequential([
        tf.keras.layers.InputLayer(input_shape=(n_features,)),
        tf.keras.layers.Dense(nodes, activation='relu'),
        tf.keras.layers.Dense(n_classes, activation='softmax')  # Use softmax for classification
    ])
    model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.01), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

def create_passive_bin_clf(nodes, n_features):
      model = tf.keras.Sequential([
          tf.keras.layers.InputLayer(input_shape=(n_features,)),
          tf.keras.layers.Dense(nodes, activation='relu'),
          tf.keras.layers.Dense(1, activation='sigmoid')  # Use softmax for classification
      ])
      model.compile(optimizer = tf.keras.optimizers.SGD(learning_rate=0.01), loss='binary_crossentropy', metrics=['accuracy'])
      return model

# Function to perform K-fold cross-validation and return the average loss
def train_passive_binary(X, y, nodes, task = "binary", n_folds=5):
    # Standardize the dataset
    scaler = StandardScaler()
    X = scaler.fit_transform(X)

    # Initialize KFold cross-validation
    kf = KFold(n_splits=n_folds, shuffle=True, random_state=42)

    # Lists to store losses for each fold
    losses = []
    print("Getting here?")

    # Loop through each fold
    for fold_idx, (train_idx, val_idx) in enumerate(kf.split(X)):
        print(f"Processing Fold {fold_idx + 1}/{n_folds}...")

        # Split data into training and validation sets
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]

        # Create the model
        n_classes = len(np.unique(y))  # Number of unique classes

        if task == "bin_clf":
          model = create_passive_bin_clf(nodes=nodes, n_features=X.shape[1])
        else:
          model = create_passive_clf_model(nodes=nodes, n_features=X.shape[1], n_classes=n_classes)

        # Early stopping callback
        early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=0)

        # Train the model
        history = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val),
                            callbacks=[early_stopping], verbose=0)

        # Evaluate on validation set and get loss
        val_loss = model.evaluate(X_val, y_val, verbose=0)[0]
        losses.append(val_loss)

    # Calculate average loss across all folds
    avg_loss = np.mean(losses)
    print(f"Average Loss across {n_folds} folds: {avg_loss:.4f}")
    return avg_loss

# Example usage:
if __name__ == "__main__":
    # Create a synthetic binary classification dataset
    X, y = make_classification(n_samples=1000, n_features=20, n_informative=15, n_classes=2, random_state=42)

    # Perform 5-fold cross-validation with a single hidden layer of 64 nodes
    #avg_loss = train_passive_binary(X, y, nodes=64, n_folds=5)


#### Practice

In [153]:
#main(sensitivity_strategy)

In [92]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer, Real
from functools import partial
from sklearn.datasets import make_regression

# Generate regression data
X, y = make_regression(n_samples=500, n_features=20, n_informative=15, noise=0.1, random_state=42)


# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 20, name='nodes'),    # Number of nodes in the hidden layer
    Integer(5, 50, name='n_queries')  # Number of queries in active learning
]

def objective(params):
    nodes, n_queries = params  # Unpack the hyperparameters (nodes, n_queries)
    query_strategy = conflicting_evidence_strategy_reg
    return main(query_strategy, X = X, y = y, task = "reg", nodes=nodes, n_queries=n_queries, init_size = 0.1)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best value for beta: {result.x[1]}")


New training with nodes 17 and n_queries 13
Processing Fold 1/5...
Iteration 1: Current Loss = inf, Min Loss = inf with pool 360
Iteration 11: Current Loss = 20.6345, Min Loss = 20.6345 with pool 230


KeyboardInterrupt: 

In [204]:
# Multiclass Classification
from skopt import gp_minimize
from skopt.space import Integer
from functools import partial

X, y = make_classification(n_samples=1000, n_features=40, n_informative=30, n_redundant=10, n_classes=4, random_state=42)

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 200, name='nodes'),    # Number of nodes in the hidden layer
    Integer(5, 50, name='n_queries')   # Number of queries in active learning
]

def objective(params):
    query_strategy = conflicting_evidence_strategy_clf
    nodes, n_queries = params  # Unpack the hyperparameters (nodes, n_queries)
    return main(query_strategy, X = X, y = y, task = "multi_clf", nodes=nodes, n_queries=n_queries, init_size = 0.1)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best number of queries: {result.x[1]}")



New training with nodes 160 and n_queries 13
Processing Fold 1/5...
(720, 40)
Iteration 1: Current Loss = 0.9227, Min Loss = inf with pool 707
Iteration 2: Current Loss = 0.9247, Min Loss = 0.9227 with pool 694
Iteration 3: Current Loss = 0.9251, Min Loss = 0.9227 with pool 681
Iteration 4: Current Loss = 0.9246, Min Loss = 0.9227 with pool 668
Iteration 5: Current Loss = 0.9044, Min Loss = 0.9227 with pool 655
Iteration 6: Current Loss = 0.9043, Min Loss = 0.9044 with pool 642
Iteration 7: Current Loss = 0.9045, Min Loss = 0.9043 with pool 629
Iteration 8: Current Loss = 0.8958, Min Loss = 0.9043 with pool 616
Iteration 9: Current Loss = 0.8897, Min Loss = 0.8958 with pool 603
Iteration 10: Current Loss = 0.8777, Min Loss = 0.8897 with pool 590
Iteration 11: Current Loss = 0.8709, Min Loss = 0.8777 with pool 577
Iteration 12: Current Loss = 0.8663, Min Loss = 0.8709 with pool 564
Iteration 13: Current Loss = 0.8645, Min Loss = 0.8663 with pool 551
Iteration 14: Current Loss = 0.8626,

Exception ignored in: <function _xla_gc_callback at 0x7ebde56200d0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/jax/_src/lib/__init__.py", line 96, in _xla_gc_callback
    def _xla_gc_callback(*args):
KeyboardInterrupt: 


Iteration 1: Current Loss = 0.9236, Min Loss = inf with pool 707
Iteration 2: Current Loss = 0.9210, Min Loss = 0.9236 with pool 694
Iteration 3: Current Loss = 0.9151, Min Loss = 0.9210 with pool 681
Iteration 4: Current Loss = 0.8894, Min Loss = 0.9151 with pool 668
Iteration 5: Current Loss = 0.8902, Min Loss = 0.8894 with pool 655
Iteration 6: Current Loss = 0.8894, Min Loss = 0.8894 with pool 642
Iteration 7: Current Loss = 0.8795, Min Loss = 0.8894 with pool 629
Iteration 8: Current Loss = 0.8783, Min Loss = 0.8795 with pool 616
Iteration 9: Current Loss = 0.8733, Min Loss = 0.8783 with pool 603
Iteration 10: Current Loss = 0.8691, Min Loss = 0.8733 with pool 590
Iteration 11: Current Loss = 0.8620, Min Loss = 0.8691 with pool 577
Iteration 12: Current Loss = 0.8593, Min Loss = 0.8620 with pool 564
Iteration 13: Current Loss = 0.8544, Min Loss = 0.8593 with pool 551
Iteration 14: Current Loss = 0.8488, Min Loss = 0.8544 with pool 538
Iteration 15: Current Loss = 0.8501, Min Loss 

#### Student Performance

##### Format

In [48]:
!pip install ucimlrepo

In [49]:
from ucimlrepo import fetch_ucirepo

In [50]:
# fetch dataset
student = fetch_ucirepo(id=320)

# data (as pandas dataframes)
X = student.data.features
y = student.data.targets

student_data = pd.concat([X, y], axis=1)

# metadata
print(student.metadata)


{'uci_id': 320, 'name': 'Student Performance', 'repository_url': 'https://archive.ics.uci.edu/dataset/320/student+performance', 'data_url': 'https://archive.ics.uci.edu/static/public/320/data.csv', 'abstract': 'Predict student performance in secondary education (high school). ', 'area': 'Social Science', 'tasks': ['Classification', 'Regression'], 'characteristics': ['Multivariate'], 'num_instances': 649, 'num_features': 30, 'feature_types': ['Integer'], 'demographics': ['Sex', 'Age', 'Other', 'Education Level', 'Occupation'], 'target_col': ['G1', 'G2', 'G3'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2008, 'last_updated': 'Fri Jan 05 2024', 'dataset_doi': '10.24432/C5TG7T', 'creators': ['Paulo Cortez'], 'intro_paper': {'ID': 360, 'type': 'NATIVE', 'title': 'Using data mining to predict secondary school student performance', 'authors': 'P. Cortez, A. M. G. Silva', 'venue': 'Proceedings of 5th Annual Future Business Technolo

In [51]:
# variable information
print(student.variables)

          name     role         type      demographic  \
0       school  Feature  Categorical             None   
1          sex  Feature       Binary              Sex   
2          age  Feature      Integer              Age   
3      address  Feature  Categorical             None   
4      famsize  Feature  Categorical            Other   
5      Pstatus  Feature  Categorical            Other   
6         Medu  Feature      Integer  Education Level   
7         Fedu  Feature      Integer  Education Level   
8         Mjob  Feature  Categorical       Occupation   
9         Fjob  Feature  Categorical       Occupation   
10      reason  Feature  Categorical             None   
11    guardian  Feature  Categorical             None   
12  traveltime  Feature      Integer             None   
13   studytime  Feature      Integer             None   
14    failures  Feature      Integer             None   
15   schoolsup  Feature       Binary             None   
16      famsup  Feature       B

In [52]:
print(sum(student_data.duplicated()))
print(student_data.shape)
student_data.head().T

0
(649, 33)


,0,1,2,3,4
school,GP,GP,GP,GP,GP
sex,F,F,F,F,F
age,18,17,15,15,16
address,U,U,U,U,U
famsize,GT3,GT3,LE3,GT3,GT3
Pstatus,A,T,T,T,T
Medu,4,1,1,4,3
Fedu,4,1,1,2,3
Mjob,at_home,at_home,at_home,health,other
Fjob,teacher,other,other,services,other


In [53]:
# Investigating data types of categorical

print(student_data['school'].value_counts())
print(student_data['sex'].value_counts())
print(student_data['address'].value_counts())
print(student_data['famsize'].value_counts())
print(student_data['Pstatus'].value_counts())
print(student_data['sex'].value_counts())
print(student_data['Mjob'].value_counts())
print(student_data['Fjob'].value_counts())
print(student_data['reason'].value_counts())
print(student_data['guardian'].value_counts())

school
GP    423
MS    226
Name: count, dtype: int64
sex
F    383
M    266
Name: count, dtype: int64
address
U    452
R    197
Name: count, dtype: int64
famsize
GT3    457
LE3    192
Name: count, dtype: int64
Pstatus
T    569
A     80
Name: count, dtype: int64
sex
F    383
M    266
Name: count, dtype: int64
Mjob
other       258
services    136
at_home     135
teacher      72
health       48
Name: count, dtype: int64
Fjob
other       367
services    181
at_home      42
teacher      36
health       23
Name: count, dtype: int64
reason
course        285
home          149
reputation    143
other          72
Name: count, dtype: int64
guardian
mother    455
father    153
other      41
Name: count, dtype: int64


In [54]:
import pandas as pd

# Assume the data is loaded into the DataFrame student_data
# Categorical columns to one-hot encode
categorical_columns = ['sex', 'school', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason', 'guardian']

# One-hot encoding the categorical variables
student_data_encoded = pd.get_dummies(student_data, columns=categorical_columns, drop_first=True)

# List of binary columns (yes/no variables)
binary_columns = ['schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher', 'internet', 'romantic']

# Map 'yes' to 1 and 'no' to 0 in the binary columns
for col in binary_columns:
    student_data_encoded[col] = student_data_encoded[col].map({'yes': 1, 'no': 0})

student_data_encoded = student_data_encoded.astype(int)

# Check the transformed dataset
print(student_data_encoded.head().T)

                    0   1   2   3   4
age                18  17  15  15  16
Medu                4   1   1   4   3
Fedu                4   1   1   2   3
traveltime          2   1   1   1   1
studytime           2   2   2   3   2
failures            0   0   0   0   0
schoolsup           1   0   1   0   0
famsup              0   1   0   1   1
paid                0   0   0   0   0
activities          0   0   0   1   0
nursery             1   0   1   1   1
higher              1   1   1   1   1
internet            0   1   1   1   0
romantic            0   0   0   1   0
famrel              4   5   4   3   4
freetime            3   3   3   2   3
goout               4   3   2   2   2
Dalc                1   1   2   1   1
Walc                1   1   3   1   2
health              3   3   3   5   5
absences            4   2   6   0   0
G1                  0   9  12  14  11
G2                 11  11  13  14  13
G3                 11  11  12  14  13
sex_M               0   0   0   0   0
school_MS   

In [55]:
print(student_data_encoded.shape)
student_X = student_data_encoded.drop(['G1', 'G2', 'G3'], axis=1)
student_y = student_data_encoded['G1']

(649, 42)


In [56]:
student_y.value_counts()

,count
G1,
10,95
11,91
12,82
13,72
14,71
9,65
8,42
15,35
7,33


In [57]:
student_y = [max(0, y - 3) for y in student_y]

In [58]:
pd.Series(student_y).value_counts()

,count
7,95
8,91
9,82
10,72
11,71
6,65
5,42
12,35
4,33
13,22


In [59]:
student_X = np.array(student_X)
student_y = np.array(student_y)

In [60]:
print(student_X.shape)
print(student_y.shape)

(649, 39)
(649,)


##### Test

In [108]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer
from functools import partial
import tensorflow as tf

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 64, name='nodes'),    # Number of nodes in the hidden layer
    Integer(5, 50, name='n_queries')   # Number of queries in active learning
]

def objective(params):
    query_strategy = conflicting_evidence_strategy_clf
    nodes, n_queries = params  # Unpack the hyperparameters (nodes, n_queries)
    return main(query_strategy, X = student_X, y = student_y, task = "multi_clf", nodes=nodes, n_queries=n_queries, init_size = 0.1)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best number of queries: {result.x[1]}")


New training with nodes 52 and n_queries 13
Processing Fold 1/5...
Iteration 1: Current Loss = 1.6423, Min Loss = inf with pool 455
Iteration 2: Current Loss = 1.5994, Min Loss = 1.6423 with pool 442
Iteration 3: Current Loss = 1.5318, Min Loss = 1.5994 with pool 429
Iteration 4: Current Loss = 1.4650, Min Loss = 1.5318 with pool 416
Iteration 5: Current Loss = 1.4024, Min Loss = 1.4650 with pool 403
Iteration 6: Current Loss = 1.3263, Min Loss = 1.4024 with pool 390
Iteration 7: Current Loss = 1.2628, Min Loss = 1.3263 with pool 377
Iteration 8: Current Loss = 1.0349, Min Loss = 1.2628 with pool 364
Iteration 9: Current Loss = 1.0140, Min Loss = 1.0349 with pool 351
Iteration 10: Current Loss = 1.0102, Min Loss = 1.0140 with pool 338
Iteration 11: Current Loss = 1.0089, Min Loss = 1.0102 with pool 327
Iteration 12: Current Loss = 1.0090, Min Loss = 1.0089 with pool 325
Iteration 13: Current Loss = 1.0074, Min Loss = 1.0089 with pool 312
Iteration 14: Current Loss = 1.0072, Min Loss =

KeyboardInterrupt: 

In [109]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer, Real
from functools import partial

X, y = make_classification(n_samples=1000, n_features=40, n_informative=30, n_redundant=10, n_classes=2, random_state=42)

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 64, name='nodes'),    # Number of nodes in the hidden layer
    Real(0.3, 0.9, name='beta')   # Number of queries in active learning
]

def objective(params):
    nodes, beta = params  # Unpack the hyperparameters (nodes, n_queries)
    query_strategy = sensitivity_strategy
    return main(query_strategy, X = student_X, y = student_y, task = "multi_clf", nodes=nodes, beta = beta)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best value for beta: {result.x[1]}")


New training with nodes 52 and beta 0.4100608739196983
Processing Fold 1/5...
Iteration 1: Current Loss = 2.5997, Min Loss = inf with pool 273


KeyboardInterrupt: 

In [62]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer, Real
from functools import partial

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 64, name='nodes'),    # Number of nodes in the hidden layer
]

def objective(nodes):
    return train_passive_binary(X = student_X, y = student_y, task = "multi", nodes=nodes[0])

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")

Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 2.4520
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 2.5517
Getting here?
Processing Fold 1/5...


...

Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 2.4935
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 2.5477
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 2.4921
Best number of nodes: 51


#### Health and Nutrition Survey

In [42]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset = fetch_ucirepo(id=887)

# data (as pandas dataframes)
X = national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.data.features
y = national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.data.targets

# metadata
print(national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.metadata)

# variable information
print(national_health_and_nutrition_health_survey_2013_2014_nhanes_age_prediction_subset.variables)


{'uci_id': 887, 'name': 'National Health and Nutrition Health Survey 2013-2014 (NHANES) Age Prediction Subset', 'repository_url': 'https://archive.ics.uci.edu/dataset/887/national+health+and+nutrition+health+survey+2013-2014+(nhanes)+age+prediction+subset', 'data_url': 'https://archive.ics.uci.edu/static/public/887/data.csv', 'abstract': "The National Health and Nutrition Examination Survey (NHANES), administered by the Centers for Disease Control and Prevention (CDC), collects extensive health and nutritional information from a diverse U.S. population. Though expansive, the dataset is often too broad for specific analytical purposes. In this sub-dataset, we narrow our focus to predicting respondents' age by extracting a subset of features from the larger NHANES dataset. These selected features include physiological measurements, lifestyle choices, and biochemical markers, which were hypothesized to have strong correlations with age.", 'area': 'Health and Medicine', 'tasks': ['Classifi

In [43]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
y = encoder.fit_transform(y)
X = np.array(X)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:114: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [40]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer
from functools import partial

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 50, name='nodes'),    # Number of nodes in the hidden layer
    Integer(10, 30, name='n_queries')   # Number of queries in active learning
]

def objective(params):
    query_strategy = conflicting_evidence_strategy_clf
    nodes, n_queries = params  # Unpack the hyperparameters (nodes, n_queries)
    return main(query_strategy, X = X, y = y, task = "bin_clf", nodes=nodes, n_queries=n_queries, init_size = 0.1)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best number of queries: {result.x[1]}")


New training with nodes 41 and n_queries 14
Processing Fold 1/5...
(1822, 7)
(1822,)
(456, 7)
(456,)


KeyboardInterrupt: 

In [135]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer, Real
from functools import partial

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 64, name='nodes'),    # Number of nodes in the hidden layer
    Real(0.3, 0.9, name='beta')   # Number of queries in active learning
]

def objective(params):
    nodes, beta = params  # Unpack the hyperparameters (nodes, n_queries)
    query_strategy = sensitivity_strategy
    return main(query_strategy, X = student_X, y = student_y, task = "multi_clf", nodes=nodes, beta = beta)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best value for beta: {result.x[1]}")

TypeError: main() got an unexpected keyword argument 'X'

In [27]:
X = np.array(X)
y = np.array(y)

In [36]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer, Real
from functools import partial

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 64, name='nodes'),    # Number of nodes in the hidden layer
]

def objective(nodes):
    return train_passive_binary(X = X, y = y, task = "binary", nodes=nodes[0])

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")

Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.2653
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.3104
Getting here?
Processing Fold 1/5...


...

Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.3116
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.3317
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.2965
Best number of nodes: 52


#### Phishing

In [45]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
phishing_websites = fetch_ucirepo(id=327)

# data (as pandas dataframes)
X = phishing_websites.data.features
y = phishing_websites.data.targets

# metadata
print(phishing_websites.metadata)

# variable information
print(phishing_websites.variables)


{'uci_id': 327, 'name': 'Phishing Websites', 'repository_url': 'https://archive.ics.uci.edu/dataset/327/phishing+websites', 'data_url': 'https://archive.ics.uci.edu/static/public/327/data.csv', 'abstract': 'This dataset collected mainly from: PhishTank archive, MillerSmiles archive, Googleâ€™s searching operators.', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 11055, 'num_features': 30, 'feature_types': ['Integer'], 'demographics': [], 'target_col': ['result'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2012, 'last_updated': 'Tue Mar 05 2024', 'dataset_doi': '10.24432/C51W2X', 'creators': ['Rami Mohammad', 'Lee McCluskey'], 'intro_paper': {'ID': 396, 'type': 'NATIVE', 'title': 'An assessment of features related to phishing websites using an automated technique', 'authors': 'R. Mohammad, F. Thabtah, L. Mccluskey', 'venue': 'International Conference for Internet Tec

In [46]:
X = np.array(X)
y = (np.array(y) + 1) / 2

In [305]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer
from functools import partial

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 264, name='nodes'),    # Number of nodes in the hidden layer
    Integer(10, 500, name='n_queries')   # Number of queries in active learning
]

def objective(params):
    query_strategy = conflicting_evidence_strategy_clf
    nodes, n_queries = params  # Unpack the hyperparameters (nodes, n_queries)
    return main(query_strategy, X = X, y = y, task = "multi_clf", nodes=nodes, n_queries=n_queries, init_size = 0.1)

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")
print(f"Best number of queries: {result.x[1]}")


New training with nodes 211 and n_queries 100
Processing Fold 1/5...
(7960, 30)
Iteration 1: Current Loss = inf, Min Loss = inf with pool 7960
Iteration 11: Current Loss = 0.1835, Min Loss = 0.1835 with pool 7148
Iteration 21: Current Loss = 0.1687, Min Loss = 0.1687 with pool 6976
Iteration 31: Current Loss = 0.1500, Min Loss = 0.1500 with pool 6827
Iteration 41: Current Loss = 0.1428, Min Loss = 0.1428 with pool 6495
Iteration 51: Current Loss = 0.1339, Min Loss = 0.1339 with pool 6051
Iteration 61: Current Loss = 0.1236, Min Loss = 0.1236 with pool 5525
Iteration 71: Current Loss = 0.1081, Min Loss = 0.1081 with pool 4624
Iteration 81: Current Loss = 0.0982, Min Loss = 0.0982 with pool 3624
Iteration 91: Current Loss = 0.0888, Min Loss = 0.0888 with pool 2624
Iteration 96: Current Loss = 0.0876, Min Loss = 0.0859
Active learning stopped after 95 iterations with loss 0.0859.
Processing Fold 2/5...
(7960, 30)
Iteration 1: Current Loss = inf, Min Loss = inf with pool 7960
Iteration 11

In [47]:
# Generic Binary Classification
from skopt import gp_minimize
from skopt.space import Integer, Real
from functools import partial

# Define the search space for Bayesian optimization (nodes and n_queries)
search_space = [
    Integer(5, 64, name='nodes'),    # Number of nodes in the hidden layer
]

def objective(nodes):
    return train_passive_binary(X = X, y = y, task = "multi", nodes=nodes[0])

# Run Bayesian optimization with Gaussian process minimization
result = gp_minimize(
    objective,             # Objective function to minimize
    search_space,          # Hyperparameter space
    n_calls=10,            # Number of iterations to run
    random_state=42
)

# Print the best hyperparameters found
print(f"Best number of nodes: {result.x[0]}")

Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.1338
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.1470
Getting here?
Processing Fold 1/5...


...

Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.1413
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.1461
Getting here?
Processing Fold 1/5...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/input_layer.py:26: UserWarning: Argument `input_shape` is deprecated. Use `shape` instead.
  warnings.warn(


Processing Fold 2/5...
Processing Fold 3/5...
Processing Fold 4/5...
Processing Fold 5/5...
Average Loss across 5 folds: 0.1319
Best number of nodes: 43


#### Mushroom: Missing

In [232]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
secondary_mushroom = fetch_ucirepo(id=848)

# data (as pandas dataframes)
X = secondary_mushroom.data.features
y = secondary_mushroom.data.targets

# metadata
print(secondary_mushroom.metadata)

# variable information
print(secondary_mushroom.variables)

{'uci_id': 848, 'name': 'Secondary Mushroom', 'repository_url': 'https://archive.ics.uci.edu/dataset/848/secondary+mushroom+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/848/data.csv', 'abstract': 'Dataset of simulated mushrooms for binary classification into edible and poisonous.', 'area': 'Biology', 'tasks': ['Classification'], 'characteristics': ['Tabular'], 'num_instances': 61068, 'num_features': 20, 'feature_types': ['Real'], 'demographics': [], 'target_col': ['class'], 'index_col': None, 'has_missing_values': 'yes', 'missing_values_symbol': 'NaN', 'year_of_dataset_creation': 2021, 'last_updated': 'Wed Apr 10 2024', 'dataset_doi': '10.24432/C5FP5Q', 'creators': ['Dennis Wagner', 'D. Heider', 'Georges Hattab'], 'intro_paper': {'ID': 259, 'type': 'NATIVE', 'title': 'Mushroom data creation, curation, and simulation to support classification tasks', 'authors': 'Dennis Wagner, D. Heider, Georges Hattab', 'venue': 'Scientific Reports', 'year': 2021, 'journal': None, '

In [234]:
y.value_counts()

,count
class,
p,33888
e,27181


#### Parkinsons: Regression

In [236]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
parkinsons_telemonitoring = fetch_ucirepo(id=189)

# data (as pandas dataframes)
X = parkinsons_telemonitoring.data.features
y = parkinsons_telemonitoring.data.targets

# metadata
print(parkinsons_telemonitoring.metadata)

# variable information
print(parkinsons_telemonitoring.variables)


{'uci_id': 189, 'name': 'Parkinsons Telemonitoring', 'repository_url': 'https://archive.ics.uci.edu/dataset/189/parkinsons+telemonitoring', 'data_url': 'https://archive.ics.uci.edu/static/public/189/data.csv', 'abstract': "Oxford Parkinson's Disease Telemonitoring Dataset", 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Tabular'], 'num_instances': 5875, 'num_features': 19, 'feature_types': ['Integer', 'Real'], 'demographics': ['Age', 'Sex'], 'target_col': ['motor_UPDRS', 'total_UPDRS'], 'index_col': ['subject#'], 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 2009, 'last_updated': 'Fri Nov 03 2023', 'dataset_doi': '10.24432/C5ZS3N', 'creators': ['Athanasios Tsanas', 'Max Little'], 'intro_paper': {'ID': 229, 'type': 'NATIVE', 'title': "Accurate Telemonitoring of Parkinson's Disease Progression by Noninvasive Speech Tests", 'authors': 'A. Tsanas, Max A. Little, P. McSharry, L. Ramig', 'venue': 'IEEE Transactions on Bio

#### Thermography: Regression

In [238]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
infrared_thermography_temperature = fetch_ucirepo(id=925)

# data (as pandas dataframes)
X = infrared_thermography_temperature.data.features
y = infrared_thermography_temperature.data.targets

# metadata
print(infrared_thermography_temperature.metadata)

# variable information
print(infrared_thermography_temperature.variables)

{'uci_id': 925, 'name': 'Infrared Thermography Temperature', 'repository_url': 'https://archive.ics.uci.edu/dataset/925/infrared+thermography+temperature+dataset', 'data_url': 'https://archive.ics.uci.edu/static/public/925/data.csv', 'abstract': 'The Infrared Thermography Temperature Dataset contains temperatures read from various locations of inferred images about patients, with the addition of oral temperatures measured for each individual. The 33 features consist of gender, age, ethnicity, ambiant temperature, humidity, distance, and other temperature readings from the thermal images. The dataset is intended to be used in a regression task to predict the oral temperature using the environment information as well as the thermal image readings. ', 'area': 'Health and Medicine', 'tasks': ['Regression'], 'characteristics': ['Tabular'], 'num_instances': 1020, 'num_features': 33, 'feature_types': ['Real', 'Categorical'], 'demographics': ['Gender', 'Age', 'Ethnicity'], 'target_col': ['aveO

In [239]:
y

,aveOralF,aveOralM
0,36.85,36.59
1,37.00,37.19
2,37.20,37.34
3,36.85,37.09
4,36.80,37.04
...,...,...
1015,36.95,36.99
1016,37.25,37.19
1017,37.35,37.59
1018,37.15,37.29


#### Testing Functions

In [63]:
import numpy as np
import tensorflow as tf
from sklearn.datasets import make_regression
import evidential_deep_learning as edl

def main():
    # Create synthetic regression data
    x_train, y_train = generate_regression_data(1000)
    x_test, y_test = generate_regression_data(1000, train=False)

    # Define the model with evidential output for regression
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(64, activation="relu", input_shape=(x_train.shape[1],)),
        tf.keras.layers.Dense(64, activation="relu"),
        edl.layers.DenseNormalGamma(1),
    ])

    # Define custom loss function with evidential regularization
    def EvidentialRegressionLoss(true, pred):
        return edl.losses.EvidentialRegression(true, pred, coeff=1e-2)

    # Early stopping callback
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)

    # Compile and train the model with early stopping
    model.compile(optimizer=tf.keras.optimizers.Adam(5e-4), loss=EvidentialRegressionLoss)
    model.fit(x_train, y_train, batch_size=100, epochs=500, validation_data=(x_test, y_test),
              callbacks=[early_stopping], verbose=0)

    # Predict on the test data
    y_pred = model.predict(x_test)

    print("Training complete.")
    return y_pred

def generate_regression_data(n_samples, train=True):
    # Use sklearn's make_regression to generate synthetic regression data
    X, y = make_regression(n_samples=n_samples, n_features=20, noise=0.1)
    X = X.astype(np.float32)
    y = y.astype(np.float32)

    if not train:
        # Add noise to test data
        y += np.random.normal(0, 3, size=y.shape).astype(np.float32)

    return X, y

y_pred = main()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step
Training complete.


In [65]:
#[mu, v, alpha, beta]
print(y_pred)

[[-1.03724585e+01  3.93922150e-04  1.00000048e+00  1.53312235e+01]
 [-6.35994244e+00  1.31807267e-03  1.00012648e+00  1.02595873e+01]
 [-7.93386745e+00  2.97468854e-04  1.00002491e+00  1.08337526e+01]
 ...
 [-7.16663361e+00  7.59767310e-04  1.00004590e+00  1.07342796e+01]
 [-6.83244658e+00  8.13805789e-04  1.00013018e+00  1.03489933e+01]
 [-7.27296925e+00  1.01219758e-03  1.00010216e+00  9.53599644e+00]]


In [ ]:
import numpy as np
import fastshap
import time

def conflicting_evidence_strategy_reg(learner, X_pool, task = "reg", n_instances=10, max = True):
    regressor = learner.estimator.model_
    preds = regressor.predict(X_pool, verbose = 0)

    # Just using alpha because measure of uncertainty of variance which is where problems are
    alpha = np.array(preds[:, 2])
    sorted_indices = np.argsort(alpha)[:n_instances]

    print(f"Indices is {sorted_indices}")

    # Return the indices and the corresponding instances
    return sorted_indices, X_pool[sorted_indices]